# Environment Set Up

In [71]:
import os
from dotenv import load_dotenv

# Loading environment variables from .env
load_dotenv()

# Changing directory to main directory for easy data access
working_directory = os.getenv("WORKING_DIRECTORY")
os.chdir(working_directory)

# Checking the change
%pwd

'/workspaces/TumorTracer'

In [72]:
from pathlib import Path

# Checking the change
print("Git folder exists:", Path(".git").exists())

Git folder exists: True


# Prediction

In [74]:
from dataclasses import dataclass
from pathlib import Path
from cnnClassifier import get_logger
from typing import Optional, Dict, Any

# Initializing the logger
logger = get_logger()

@dataclass(frozen=True)
class PredictionConfig:
    """
    Immutable configuration class to store all parameters 
    and paths required for model prediction. 
    """
    trained_model_path: Path                     # Path to the trained model that will be used to predict
    class_indices_path: Path                    # Path to the model's class_indices
    params_image_size: tuple[int, int, int]     # Input image size, e.g., [224, 224, 3]
    params_normalization: float 				# Normalization factor used in training

In [78]:
from cnnClassifier.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from cnnClassifier.utils.common import read_yaml, create_directories
from cnnClassifier import get_logger

# Initializing the logger
logger = get_logger()

class ConfigurationManager:
    def __init__(self, config_file_path=CONFIG_FILE_PATH, params_file_path=PARAMS_FILE_PATH) -> None:
        """
        Reads configuration files (config.yaml and params.yaml), 
        ensures necessary directories exist, and prepares structured config objects.

        Args:
        - config_file_path (str): Path to the config.yaml file.
        - params_file_path (str): Path to the params.yaml file.
        """
        # Validate and load config.yaml
        if not Path(config_file_path).exists():
            logger.error(f"Config file not found at: {config_file_path}")
            raise FileNotFoundError(f"Config file not found at: {config_file_path}")
        self.config = read_yaml(config_file_path)

        # Validate and load params.yaml
        if not Path(config_file_path).exists():
            logger.error(f"Params file not found at: {params_file_path}")
            raise FileNotFoundError(f"Params file not found at: {params_file_path}")
        self.params = read_yaml(params_file_path)

        logger.info(f"Loading configuration from {config_file_path} and parameters from {params_file_path}")

        # Create the root artifacts directory (if not already present)
        create_directories([self.config.artifacts_root])


    def get_prediction_config(self) -> PredictionConfig:
        """
        Prepares and returns the PredictionConfig object.

        Returns:
        - PredictionConfig: Structured config for predicting using trained model
        """
        config = self.config.predictions
        params = self.params.predictions

        predictions_config = PredictionConfig(
            trained_model_path=Path(config.model),
            class_indices_path=Path(config.class_indices),
            params_image_size=tuple(params.IMAGE_SIZE),
            params_normalization=params.NORMALIZATION,
        )

        logger.info(f"PredictionConfig created with: {predictions_config}")

        return predictions_config

In [ ]:
import random
import numpy as np
import tensorflow as tf

from typing import Union
from tensorflow.keras.preprocessing import image  # type:ignore
from pathlib import Path
from collections import defaultdict
from tqdm import tqdm

from cnnClassifier.utils.common import load_json
from cnnClassifier import get_logger

# Initializing the logger
logger = get_logger()

class Predictions:
	"""
	Wrapper for loading a trained model and generating predictions on input images.

	Responsibilities:
	- Load a trained Keras model and class indices.
	- Preprocess input images for inference.
	- Predict the class label for a given image.
	"""
	def __init__(self, config: PredictionConfig) -> None:
		self.config = config
		self.model = self._load_model()
		self.classes = self._get_classes()

		# Set random seeds
		seed = self.config.params_seed if hasattr(self.config, "params_seed") else 1234
		tf.random.set_seed(seed)
		np.random.seed(seed)
		random.seed(seed)


	def predict(self, image_path: Union[str, Path], verbose: bool = True) -> str:
		"""
		Predict the class label for a single image.

		Args:
        - image_path (str | Path): Path to the image file.

		Returns:
        - str: Predicted class label.
		"""
		try:
			prediction_label, _ = self.predict_with_confidence(image_path=image_path, verbose=verbose)
			return prediction_label

		except Exception as exception_error:
			logger.error(f"Unexpected while trying to predict: {exception_error}")
			raise 

     
	def predict_with_confidence(self, image_path: Union[str, Path], verbose: bool = True) -> tuple[str, float]:
		"""
		Predict the class label for a single image, and it confidence

		Args:
        - image_path (str | Path): Path to the image file.

		Returns:
        - str: Predicted class label.
		- float: Confidence in prediction.
		"""
		try:
			image_path = Path(image_path)
			if not image_path.exists():
				raise FileNotFoundError(f"Image file not found: {image_path}")

			image_object = image.load_img(image_path, target_size=self.config.params_image_size[0:2])
			image_array = image.img_to_array(image_object) / self.config.params_normalization

			# Add batch dimension
			image_dims = np.expand_dims(image_array, axis=0)

			# Predicting image
			prediction = self.model.predict(image_dims, verbose=0)
			prediction_idx = np.argmax(prediction, axis=1)[0]
			prediction_label = self.classes[prediction_idx]

			confidence = prediction[0][prediction_idx]

			if verbose:
				logger.info(f"Predicted '{prediction_label}' for image: {image_path}")

			return prediction_label, confidence
		
		except Exception as exception_error:
			logger.error(f"Unexpected while trying to predict: {exception_error}")
			raise 


	def evaluate(self, image_directory: Union[str, Path], verbose: bool = False) -> dict[str, float]:
		"""
		Evaluates model performance on a labeled image dataset organized in subfolders by class.

		Args:
		- image_directory (str | Path): Root directory containing subfolders (each representing a class).

		Returns:
		- dict[str, float]: Dictionary containing per-class accuracy and overall accuracy as "OVERALL".
		"""
		image_directory_path = Path(image_directory)

		if not image_directory_path.exists():
			logger.error(f"Provided dataset directory does not exist: {image_directory_path}")
			raise FileNotFoundError(f"Provided dataset directory does not exist: {image_directory_path}")
		
		correct_per_class = defaultdict(int)
		total_per_class = defaultdict(int)

		for subfolder in tqdm(list(image_directory_path.iterdir()), desc=f"Evaluating Subfolders", leave=True):
			if not subfolder.is_dir():
				continue
			
			true_label = subfolder.name

			for image in list(subfolder.glob("*")):
				if not image.suffix.casefold() in [".jpg", ".jpeg", ".png"]:
					continue

				try:
					predicted_label = self.predict(image_path=image, verbose=verbose)
					if predicted_label.casefold() == true_label.casefold():
						correct_per_class[true_label] += 1
					total_per_class[true_label] += 1

				except Exception as exception_error:
					logger.warning(f"Skipping {image.name}: {exception_error}")
					continue

		result = {}

		# Overall accuracy
		total_correct = sum(correct_per_class.values())
		total_images = sum(total_per_class.values())
		result["Overall"] = total_correct / total_images if total_images > 0 else 0.0

		# Per-class accuracy
		for class_name in total_per_class:
			result[class_name] = correct_per_class[class_name] / total_per_class[class_name] if total_images > 0 else 0.0

		logger.info(f"Evaluation Summary: {str(result)}")
		return result


	def _load_model(self) -> tf.keras.Model:
		"""
		Loads the trained model from disk.

		Returns:
		- tf.keras.Model: Loaded Keras model.
		"""
		model_path = Path(self.config.trained_model_path)

		if not model_path.exists():
			logger.error(f"Could not find model at {model_path}.")
			raise FileNotFoundError(f"Could not find model at {model_path}.")
		
		try:
			output_model = tf.keras.models.load_model(model_path)
			logger.info(f"Successfully loaded the base model from {model_path}.")
			return output_model

		except Exception as exception_error:
			logger.error(f"Unexpected error while loading the update base model at {model_path}: {exception_error}")
			raise 
			

	def _get_classes(self) -> dict[int, str]:
		"""
		Loads and inverts class indices from JSON file.

        Returns:
		- dict: Mapping from index to class label.
		"""
		try:
			classes = load_json(self.config.class_indices_path)
			return {value: key for key, value in classes.items()}
		
		except Exception as exception_error:
			logger.error(f"Unexpected error while loading class indicies: {exception_error}")
			raise 

In [113]:
try:
	config_manager = ConfigurationManager()
	prediction_config = config_manager.get_prediction_config()

	prediction_constructor = Predictions(config=prediction_config)
	prediction_constructor.predict(image_path="artifacts/data_ingestion/Data/train/normal/2 - Copy - Copy.png")

except Exception as exception_error:
	logger.exception(f"Unexpected error during prediction: {exception_error}")
	raise

[2025-07-09 11:28:12,589: INFO: common: YAML file: config/config.yaml loaded successfully]
[2025-07-09 11:28:12,597: INFO: common: YAML file: params/params.yaml loaded successfully]
[2025-07-09 11:28:12,598: INFO: 858813328: Loading configuration from config/config.yaml and parameters from params/params.yaml]
[2025-07-09 11:28:12,599: INFO: common: Directory: artifacts created successfully.]
[2025-07-09 11:28:12,600: INFO: 858813328: PredictionConfig created with: PredictionConfig(trained_model_path=PosixPath('artifacts/model_training/trained_model_20250708_0949.keras'), class_indices_path=PosixPath('artifacts/model_training/class_indices.json'), params_image_size=(224, 224, 3), params_normalization=255.0)]
[2025-07-09 11:28:12,939: INFO: 2296268337: Successfully loaded the base model from artifacts/model_training/trained_model_20250708_0949.keras.]
[2025-07-09 11:28:12,940: INFO: common: JSON file succesfully loaded form: artifacts/model_training/class_indices.json]
[2025-07-09 11:28:

In [114]:
try:
	config_manager = ConfigurationManager()
	prediction_config = config_manager.get_prediction_config()
	
	prediction_constructor = Predictions(config=prediction_config)
	prediction_constructor.evaluate(image_directory="artifacts/data_ingestion/Data/valid/")

except Exception as exception_error:
	logger.exception(f"Unexpected error during prediction evaluation: {exception_error}")
	raise

[2025-07-09 11:28:13,627: INFO: common: YAML file: config/config.yaml loaded successfully]
[2025-07-09 11:28:13,630: INFO: common: YAML file: params/params.yaml loaded successfully]
[2025-07-09 11:28:13,632: INFO: 858813328: Loading configuration from config/config.yaml and parameters from params/params.yaml]
[2025-07-09 11:28:13,633: INFO: common: Directory: artifacts created successfully.]
[2025-07-09 11:28:13,634: INFO: 858813328: PredictionConfig created with: PredictionConfig(trained_model_path=PosixPath('artifacts/model_training/trained_model_20250708_0949.keras'), class_indices_path=PosixPath('artifacts/model_training/class_indices.json'), params_image_size=(224, 224, 3), params_normalization=255.0)]
[2025-07-09 11:28:14,017: INFO: 2296268337: Successfully loaded the base model from artifacts/model_training/trained_model_20250708_0949.keras.]
[2025-07-09 11:28:14,018: INFO: common: JSON file succesfully loaded form: artifacts/model_training/class_indices.json]


Evaluating Subfolders: 100%|██████████| 4/4 [00:44<00:00, 11.24s/it]

[2025-07-09 11:28:58,978: INFO: 2296268337: Evaluation Summary: {'Overall': 0.7083333333333334, 'adenocarcinoma': 0.8260869565217391, 'normal': 0.8461538461538461, 'large_cell_carcinoma': 0.5238095238095238, 'squamous_cell_carcinoma': 0.6666666666666666}]
